In [131]:
# libraries
import numpy as np
import torch
import torch.nn.functional as F
import cv2
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import os


In [ ]:
# device use CUDA
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# hyperparameters
random_seed = 1
learning_rate = 0.001
num_epochs = 100
batch_size = 256

# architecture
num_classes = 2

In [ ]:
# visualizing the neuroimages

# if the file is a directory, we need to use os.listdir() to get the .dcm files inside the directory
# pydicom.dcmread is trying to read a file and not a directory which our file is.

import pydicom
import matplotlib.pyplot as plt
import os
import random

random_int = random.randint(0,2000)

# for mri normal images
dicom_MRI_Norm_dir = "C:/Users/jerem/Downloads/MRI_Dataset/unzipped/Extracted_MRI_DCM/MRI_Norm"
dicom_MRI_Norm_files = [f for f in os.listdir(dicom_MRI_Norm_dir) if f.endswith(".dcm")]

if dicom_MRI_Norm_files:
    dicom_file = os.path.join(dicom_MRI_Norm_dir , dicom_MRI_Norm_files[random_int])
    dataset = pydicom.dcmread(dicom_file)

    pixel_array = dataset.pixel_array

    if len(pixel_array.shape) == 2:
        # Case 1: 2D Image
        plt.imshow(dataset.pixel_array, cmap=plt.cm.bone)

    elif len(pixel_array.shape) == 3:  
        # Case 2: 3D Image (Select the middle slice)
        middle_slice = pixel_array.shape[0] // 2  
        plt.imshow(pixel_array[middle_slice], cmap=plt.cm.bone)

    plt.title("Random MRI Normal DICOM Image")
    plt.axis("off")
    image_shape = dataset.pixel_array.shape
    print("Original MRI Normal Image Shape:", image_shape)
    plt.show()
else:
    print("No DICOM files found in the directory!")


# for mri ad images
dicom_MRI_AD_dir = "C:/Users/jerem/Downloads/MRI_Dataset/unzipped/Extracted_MRI_DCM/MRI_AD"
dicom_MRI_AD_files = [f for f in os.listdir(dicom_MRI_AD_dir) if f.endswith(".dcm")]

if dicom_MRI_AD_files:
    dicom_file = os.path.join(dicom_MRI_AD_dir , dicom_MRI_AD_files[random_int])
    dataset = pydicom.dcmread(dicom_file)

    pixel_array = dataset.pixel_array

    if len(pixel_array.shape) == 2:
        # Case 1: 2D Image
        plt.imshow(dataset.pixel_array, cmap=plt.cm.bone)

    elif len(pixel_array.shape) == 3:  
        # Case 2: 3D Image (Select the middle slice)
        middle_slice = pixel_array.shape[0] // 2  
        plt.imshow(pixel_array[middle_slice], cmap=plt.cm.bone)

    plt.title("Random MRI AD DICOM Image")
    plt.axis("off")
    image_shape = dataset.pixel_array.shape
    print("Original MRI AD Image Shape:", image_shape)
    plt.show()
else:
    print("No DICOM files found in the directory!")


In [ ]:
image_shape = dataset.pixel_array.shape
print("Original DICOM Image Shape:", image_shape)

In [ ]:
# visualizing neuroimages

random_int = random.randint(0,2000)

# for pet normal images
dicom_PET_Norm_dir = "C:/Users/jerem/Downloads/PET_Dataset/unzipped/Extracted_PET_DCM/PET_Norm"
dicom_PET_Norm_files = [f for f in os.listdir(dicom_PET_Norm_dir) if f.endswith(".dcm")]

if dicom_PET_Norm_files:
    dicom_file = os.path.join(dicom_PET_Norm_dir , dicom_PET_Norm_files[random_int])
    dataset = pydicom.dcmread(dicom_file)

    pixel_array = dataset.pixel_array

    if len(pixel_array.shape) == 2:
        # Case 1: 2D Image
        plt.imshow(dataset.pixel_array, cmap=plt.cm.bone)

    elif len(pixel_array.shape) == 3:  
        # Case 2: 3D Image (Select the middle slice)
        middle_slice = pixel_array.shape[0] // 2  
        plt.imshow(pixel_array[middle_slice], cmap=plt.cm.bone)

    plt.title("Random PET Normal DICOM Image")
    plt.axis("off")
    image_shape = dataset.pixel_array.shape
    print("Original PET Normal Image Shape:", image_shape)
    plt.show()
else:
    print("No DICOM files found in the directory!")


# for pet ad images
dicom_PET_AD_dir = "C:/Users/jerem/Downloads/PET_Dataset/unzipped/Extracted_PET_DCM/PET_AD"
dicom_PET_AD_files = [f for f in os.listdir(dicom_PET_AD_dir) if f.endswith(".dcm")]

if dicom_PET_AD_files:
    dicom_file = os.path.join(dicom_PET_AD_dir , dicom_PET_AD_files[random_int])
    dataset = pydicom.dcmread(dicom_file)

    pixel_array = dataset.pixel_array

    if len(pixel_array.shape) == 2:
        # Case 1: 2D Image
        plt.imshow(dataset.pixel_array, cmap=plt.cm.bone)

    elif len(pixel_array.shape) == 3:  
        # Case 2: 3D Image (Select the middle slice)
        middle_slice = pixel_array.shape[0] // 2  
        plt.imshow(pixel_array[middle_slice], cmap=plt.cm.bone)

    plt.title("Random PET AD DICOM Image")
    plt.axis("off")
    image_shape = dataset.pixel_array.shape
    print("Original PET AD Image Shape:", image_shape)
    plt.show()
else:
    print("No DICOM files found in the directory!")

In [ ]:
import os
import glob
import numpy as np
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms

base_dir = "C:/Users/jerem/Desktop/MRI_PET_Dataset"
categories = {
    "MRI_Norm": 0, "MRI_AD": 1,
    "PET_Norm": 0, "PET_AD": 1
}

data = []
for category, label in categories.items():
    files = glob.glob(os.path.join(base_dir, category, "*.dcm"))
    data.extend([(f, label) for f in files])

np.random.shuffle(data)

# Split into 80% train, 20% test
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Further split train into 90% test, 10% validation
test_data, val_data = train_test_split(test_data, test_size=0.1, random_state=42)

print(f"Train: {len(train_data)}, Validation: {len(val_data)}, Test: {len(test_data)}")


In [ ]:
class BrainScanDataset(dataset):
    def __init__(self, data_list, transform=None):
        self.data_list = data_list
        self.transform = transform

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        file_path, label = self.data_list[idx]
        
        dicom_image = pydicom.dcmread(file_path).pixel_array
        
        image = torch.tensor(dicom_image, dtype=torch.float32).unsqueeze(0)

        if self.transform:
            image = self.transform(image)
            
        return image, torch.tensor(label, dtype=torch.long)

In [ ]:

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.GaussianBlur(kernel_size=5, sigma=1.5),
    transforms.Normalize(mean=[0.5], std=[0.5]) 
])

train_dataset = BrainScanDataset(train_data, transform=transform)
val_dataset = BrainScanDataset(val_data, transform=transform)
test_dataset = BrainScanDataset(test_data, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for images, labels in train_loader:
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)  
    break 

In [ ]:
# CNN model
class CNN_model(torch.nn.Module):
    def __init__(self, num_classes):
        super(CNN_model, self).__init__()

        self.conv_1 = torch.nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=0)

        self.maxpool_1 = torch.nn.MaxPool2d(2, 2)


        self.conv_2 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0) 

        self.maxpool_2 = torch.nn.MaxPool2d(2, 2)


        self.conv_3 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=0) 

        self.maxpool_3 = torch.nn.MaxPool2d(2, 2)


        self.conv_4 = torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=1, padding=0) 

        self.maxpool_4 = torch.nn.MaxPool2d(2, 2)


        self.fc1 = torch.nn.Linear(128 * 2 * 2, 1024)

        self.fc2 = torch.nn.Linear(1024, num_classes)

        

    def forward(self, x):
        out = F.relu(self.conv_1(x))
        out = self.maxpool_1(out)

        out = F.relu(self.conv_2(x))
        out = self.maxpool_2(out)

        out = F.relu(self.conv_3(x))
        out = self.maxpool_3(out)

        out = F.relu(self.conv_4(x))
        out = self.maxpool_4(out)

        out = out.view(-1, 128 * 2 * 2)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)

        return out

In [ ]:
# training CNN model
torch.manual_seed(random_seed)

model = CNN_model(num_classes=num_classes).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

criterion = torch.nn.CrossEntropyLoss()

start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (features, targets) in enumerate(train_loader):
        features, targets = features.to(device), targets.to(device)

        optimizer.zero_grad()
        
        outputs = model(features)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if batch_idx % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 100))
            running_loss = 0.0

print('CNN Training Finished')

In [ ]:
# compute accuracy of model
def compute_accuracy(model, data_loader):
    correct=0
    total=0

    for batch_idx, (features, targets) in enumerate(data_loader):
        features, targets = features.to(device), targets.to(device)

        outputs = model(features)
        _, predicted = torch.max(outputs, 1)

        correct += (predicted == targets).sum().item()
        total += targets.size(0)

    acc = (correct/total) * 100
    
    return acc

In [ ]:
# compute sensitivity (recall)
def compute_sensitivity(model, data_loader):
    truePositive=0
    falseNegative=0

    for batch_idx, (features, targets) in enumerate(data_loader):
        features, targets = features.to(device), targets.to(device)

        outputs = model(features)
        _, predicted = torch.max(outputs, 1)

        truePositive += ((predicted == 1) & (targets == 1)).sum().item()
        falseNegative += ((predicted == 0) & (targets == 1)).sum().item()

    sens = truePositive / (truePositive + falseNegative)

    return sens

In [ ]:
# compute specificity 
def compute_specificity(model, data_loader):
    trueNegative=0
    falsePositive=0

    for batch_idx, (features, targets) in enumerate(data_loader):
        features, targets = features.to(device), targets.to(device)

        outputs = model(features)
        _, predicted = torch.max(outputs, 1)

        trueNegative += ((predicted == 0) & (targets == 0)).sum().item()
        falsePositive += ((predicted == 1) & (targets == 0)).sum().item()

    spec = trueNegative / (trueNegative + falsePositive)

    return spec

In [ ]:
# compute f1-score
def compute_f1_score(model, data_loader):
    truePositive=0
    trueNegative=0
    falsePositive=0
    falseNegative=0

    for batch_idx, (features, targets) in enumerate(data_loader):
        features, targets = features.to(device), targets.to(device)

        outputs = model(features)
        _, predicted = torch.max(outputs, 1)

        truePositive += ((predicted == 1) & (targets == 1)).sum().item()
        trueNegative += ((predicted == 0) & (targets == 0)).sum().item()

        falsePositive += ((predicted == 1) & (targets == 0)).sum().item()
        falseNegative += ((predicted == 0) & (targets == 1)).sum().item()

        # precison = TP / (TP + FP)
        precision = truePositive / (truePositive + falseNegative)

        # sensitivity = TP / (TP + FN)
        sensitivity = truePositive / (truePositive + falseNegative)

    f1 = (2 * precision * sensitivity) / (precision + sensitivity)

    return f1

In [ ]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))
print('Test sensitivity: %.2f%%' % (compute_sensitivity(model, test_loader)))
print('Test specificity: %.2f%%' % (compute_specificity(model, test_loader)))
print('Test f1-score: %.2f%%' % (compute_f1_score(model, test_loader)))